# Determine the frequency and cause of contradicting highscores
**Contributors:** Victor Lin

**Achievement:** The frequency of duplicate scores (scores from the same player, beatmap, and mods) was found to be 778, or ~.00078% of the 10M scores present across all dumps. Duplicates can be safely ignored for the data cleaning process.

**Requirements:**

1. *exploration/sql_migration/random_dump_migration.ipynb*

## Introduction
Similar to the exploration of user crossover between random dumps, this notebook takes a closer look at contradicting user highscores, for the same beatmap and dump, found in *random_dump_migration.ipynb*

In [3]:
import sys
sys.path.append('../..')
from exploration.config import sql_inst, mongo_inst

In [4]:
osu_random_db = mongo_inst['osu_random_db']

## Contradicting Highscores Pipeline Filter
Define a pipeline that groups highscores by identical (user_id, beatmap_id, and mods) index. Filter for groups with more than one highscore found (aka, contradicting highscores)

In [5]:
pipeline = [
    {
        '$group': {
            '_id': {
                'user_id': '$user_id',
                'beatmap_id': '$beatmap_id',
                'enabled_mods': '$enabled_mods'
            },
            'count': {
                
                '$sum': 1
            }
        }
    },
    {
        '$match': {
            'count': {
                '$gt': 1
            }
        }
    },

]

duplicate_indecies = osu_random_db['osu_scores_high'].aggregate(pipeline, allowDiskUse = True)

## Querying highscores from each multi-index
For the Aug - Jan dumps, it appears there are only cases of 2 contradicting highscores for each index (as opposed to 3 or more)

In [15]:
duplicate_highscores = []

for duplicate_index in duplicate_indecies:
    index = duplicate_index['_id']
    duplicate_highscores.extend(osu_random_db['osu_scores_high'].find(index, {'mlpp': 0}))

## Investigating Highscores with DataFrame
Reordered table columns to have the 3 indexed columsn (beatmap_id, user_id, enabled_mods) to be first

In [24]:
import pandas as pd
df = pd.DataFrame(duplicate_highscores)
df.set_index(['beatmap_id', 'user_id', 'enabled_mods'])

cols = df.columns.tolist()
cols = cols[1:3] + cols[-6:-5] + cols[:1] + cols[3: -6] + cols[-5:]
df = df[cols]

## Tables for first 5 contradicting pairs of highscores
It appears that for each pair, one highscore was submitted in 2011 or 2012 and one in 2017. Perhaps there was a change to score storage in this time that occasionally missed duplicates.

In [46]:
from IPython.display import display

for i in range(5):
    print(f'\n\nContradicting highscores GROUP {i + 1}')
    display(df[2 * i: 2 * (i + 1)])



Contradicting highscores GROUP 1


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
0,82249,125326,0,2352651667,1924818,239,B,6,39,420,4,67,15,0,2017-08-16 05:44:25,24.0668,0,0,NL
1,82249,125326,0,819090105,7711092,672,S,0,23,446,0,74,15,0,2012-08-09 16:44:31,69.1838,0,0,NL




Contradicting highscores GROUP 2


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
2,34208,222108,0,2405680539,2555898,438,A,1,14,267,1,46,11,0,2017-11-15 10:53:20,32.6380,0,0,BE
3,34208,222108,0,414144947,1135712,236,B,7,43,229,4,33,18,0,2011-08-24 10:58:25,15.3665,0,0,BE




Contradicting highscores GROUP 3


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
4,31819,313045,0,2317893925,11367840,602,A,10,57,895,6,41,17,0,2017-06-14 20:31:29,68.2835,0,0,CO
5,31819,313045,0,227036885,24931740,1145,S,3,69,896,0,43,19,0,2011-01-30 15:19:55,124.2720,0,0,CO




Contradicting highscores GROUP 4


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
6,120695,313045,0,638942003,4283606,338,A,0,27,659,6,161,15,0,2012-03-24 13:27:01,70.4094,0,0,CO
7,120695,313045,0,2261310360,10552114,756,A,0,16,674,2,163,14,0,2017-03-04 22:28:55,149.9820,0,0,CO




Contradicting highscores GROUP 5


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
8,63499,330565,0,387012375,9377320,601,S,1,24,478,0,74,16,1,2011-07-29 09:02:27,99.9510,0,0,CN
9,63499,330565,0,2331103163,9260730,601,S,0,34,469,0,73,18,1,2017-07-09 07:08:13,92.3771,0,0,CN


## Tables for last 5 contradicting pairs of highscores
The score _ids for the first 4 pairs are extremely close, and in the 3rd and 4th pair the other columns are the exact same. This may be an unintentional double submission on the serverside. Overall, contradicting highscores appear to be due to server-side errors, and we should simply choose the higher score.

In [53]:
N = len(duplicate_indecies)
for i in range(N - 5, N):
    print(f'\n\nContradicting highscores GROUP {i + 1}')
    display(df[2 * i: 2 * (i + 1)])



Contradicting highscores GROUP 147


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
292,1257655,16367408,16,3116695797,526492,206,A,2,21,148,0,16,14,0,2020-06-09 19:42:04,8.17476,0,0,MX
293,1257655,16367408,16,3116695798,502325,169,A,2,15,154,0,18,12,0,2020-06-09 19:42:04,7.24461,0,0,MX




Contradicting highscores GROUP 148


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
294,821752,16576576,0,3040973988,32536,36,C,5,16,40,2,8,10,0,2020-04-02 09:35:49,0.402358,0,0,US
295,821752,16576576,0,3040973986,36122,64,D,7,18,37,1,9,7,0,2020-04-02 09:35:49,0.640620,0,0,US




Contradicting highscores GROUP 149


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
296,1884614,16576870,0,3045971177,668500,162,C,8,54,239,7,45,28,0,2020-04-07 09:55:58,10.7168,1,0,RU
297,1884614,16576870,0,3045971178,668500,162,C,8,54,239,7,45,28,0,2020-04-07 09:55:58,10.7168,1,0,RU




Contradicting highscores GROUP 150


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
298,2279985,16630180,0,3048885837,598556,226,X,0,0,133,0,27,0,1,2020-04-10 06:46:29,16.2466,0,0,ID
299,2279985,16630180,0,3048885838,598556,226,X,0,0,133,0,27,0,1,2020-04-10 06:46:29,16.2466,0,0,ID




Contradicting highscores GROUP 151


,beatmap_id,user_id,enabled_mods,_id,score,maxcombo,rank,count50,count100,count300,countmiss,countgeki,countkatu,perfect,date,pp,replay,hidden,country_acronym
300,1937471,18927678,1,3297712219,36834,52,C,2,14,55,1,12,10,0,2020-10-28 03:27:58,1.50850,0,0,FR
301,1937471,18927678,1,3297715568,37412,48,C,0,14,57,1,14,10,0,2020-10-28 03:31:50,1.44036,0,0,FR
